In [1]:
# # import librarie
! pip install --upgrade pip

! pip install --user pandas
! pip install --user scipy
! pip install --user numpy
! pip install --user seaborn
! pip install --user tqdm
! pip install --user spotipy
! pip install --user scikit-learn
# ! pip install --user sklearn.preprocessing


# ! pip install --user sklearn

# from sklearn.preprocessing import MinMaxScaler
# ! pip install --user <package>
# ! pip install --user <package>
# ! pip install --user <package>
import os
import json
import operator
import sklearn
import numpy as np
import statistics
import pandas as pd
import scipy.stats as stats
import spotipy
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

Matplotlib created a temporary cache directory at /var/folders/sp/cyhd19411ndb804v36t6lxy80000gn/T/matplotlib-nh23h0hi because the default path (/Users/mileslow/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# define global variables
SIZE = 500 # this defines how many playlists we want

# for Spotify API
cid = 'a0d7b6c810324590b2992e0b5d024059'
secret = 'b02ba0652fe64f568634d0e5c15ab422'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [3]:
def unpack(json_name):
    '''
    unpack a json playlist file to obtain a playlist
    input: a file name
    return: SIZE list of playlists
    '''
    # Opening JSON file
    f = open("playlists.json")

    # returns JSON object as
    # a dictionary
    data = json.load(f)
    f.close()
    playlists = data['playlists']
    return playlists

playlists = unpack(json_name='playlists.json')

In [4]:
def find_uris(playlists, start=0, SIZE=SIZE):
    '''
    ouput the uri list for a given playlist
    input: a playlist
    return: a list of all track uris, and the playlist uri
    '''
    track_uris = [[i['track_uri'] for i in playlists[j]['tracks']] for j in range(start, SIZE)]
    pids = [playlists[i]['pid'] for i in range(start, SIZE)]
    return track_uris, pids

# check track_uris for first playlist in dataset
start=0
uri_list, pids = find_uris(playlists, start, SIZE)

In [5]:
def playlist_summarise(playlist_uri):
    '''
    where we query playlist uris with spotify API
    input: list of uris for a given playlist
    return: the mean features of the given playlist
    '''
    all_key = np.zeros(len(playlist_uri))
    all_acousticness = np.zeros(len(playlist_uri))
    all_danceability = np.zeros(len(playlist_uri))
    all_energy = np.zeros(len(playlist_uri))
    all_instrumentalness = np.zeros(len(playlist_uri))
    all_loudness = np.zeros(len(playlist_uri))
    all_speechiness = np.zeros(len(playlist_uri))
    all_valence = np.zeros(len(playlist_uri))
    all_tempo = np.zeros(len(playlist_uri))

    # unpack each uri
    for i in tqdm(range(len(playlist_uri))):
        # query spotify api
        audio_features = sp.audio_features(playlist_uri[i])
        all_key[i] = audio_features[0]['key']
        all_acousticness[i] = audio_features[0]['acousticness']
        all_danceability[i] = audio_features[0]['danceability']
        all_energy[i] = audio_features[0]['energy']
        all_instrumentalness[i] = audio_features[0]['instrumentalness']
        all_loudness[i] = audio_features[0]['loudness']
        all_speechiness[i] = audio_features[0]['speechiness']
        all_valence[i] = audio_features[0]['valence']
        all_tempo[i] = audio_features[0]['tempo']

    # calculate means
    key = np.mean(all_key)
    acousticness = np.mean(all_acousticness)
    danceability = np.mean(all_danceability)
    energy = np.mean(all_energy)
    instrumentalness = np.mean(all_instrumentalness)
    loudness = np.mean(all_loudness)
    speechiness = np.mean(all_speechiness)
    valence = np.mean(all_valence)
    tempo = np.mean(all_tempo)

    # return all means
    return [key, acousticness, danceability, energy,
            instrumentalness, loudness, speechiness,
            valence, tempo]

In [6]:
def normalize_df(df, col_names):
    x = df.values #returns a numpy array
    min_max_scaler = MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns=col_names)
    return df

In [7]:
def create_playlist_dataframe(playlists):
    '''
    summary function to allow ease of playlist transformation into a df
    input: SIZE list of playlists
    output: dataframe with all mean playlist features
    '''
    # find uris and playlist ids
    uri_list, pids = (playlists, start, SIZE)

    # set up dataframe
    col_names = ['pid', 'key', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'valence', 'tempo']
    df = pd.DataFrame(columns=col_names) # generate empty df

    # iterate through and get features for each playlist
    for i in range(SIZE):
        features = playlist_summarise(uri_list[i])
        features.insert(0, 0)
        df.loc[i] = features

    df = normalize_df(df, col_names)
    # insert ids
    df['pid'] = pids
    return df

In [8]:
#playlist_df = create_playlist_dataframe(playlists)
playlist_df = pd.read_csv('playlist_df.csv', index_col=0)
playlist_df

,pid,key,acousticness,danceability,energy,instrumentalness,loudness,speechiness,valence,tempo
0,834000,0.645308,0.173219,0.857485,0.608134,0.015506,0.806802,0.209173,0.326380,0.395954
1,834001,0.757040,0.451602,0.480572,0.532890,0.389105,0.654947,0.010219,0.499174,0.353239
2,834002,0.679062,0.098866,0.760773,0.782856,0.010693,0.903573,0.048531,0.634635,0.427712
3,834003,0.567796,0.010496,0.433334,0.965381,0.000039,0.960515,0.059296,0.593296,0.697373
4,834004,0.631160,0.413368,0.551870,0.498053,0.027596,0.710757,0.009369,0.387782,0.449970
...,...,...,...,...,...,...,...,...,...,...
495,834495,0.652280,0.291041,0.528743,0.521976,0.043901,0.686463,0.005384,0.412935,0.413829
496,834496,0.390438,0.380231,0.456496,0.652084,0.000217,0.818805,0.055452,0.462519,0.704571
497,834497,0.482072,0.375387,0.580396,0.662425,0.134960,0.782003,0.041677,0.553171,0.431537
498,834498,0.668866,0.220879,0.600123,0.736781,0.029747,0.852047,0.077878,0.539488,0.491324


In [9]:
# convert genre to label encoded
genres = ['pop', 'hip-hop', 'edm', 'latin', 'rock',
          'r-n-b', 'country', 'jazz', 'classical',
          'alternative']

In [10]:
# number of songs to query per genre
song_num = 100
n_requests = 20

In [11]:
# we generate using genre seeds
pop_uris = []
hip_hop_uris = []
edm_uris = []
latin_uris = []
rock_uris = []
randb_uris = []
country_uris = []
jazz_uris = []
classical_uris = []
alternative_uris = []

# query spotify api for each genre n_requests time
# this method bypasses the 100 limit on song queries
for i in range(n_requests):
    print(i)
    pop_recs = sp.recommendations(seed_genres=['pop'], limit=song_num)
    pop_uris += [i['uri'] for i in pop_recs['tracks']]
    hip_hop_recs = sp.recommendations(seed_genres=['hip-hop'], limit=song_num)
    hip_hop_uris += [i['uri'] for i in hip_hop_recs['tracks']]
    edm_recs = sp.recommendations(seed_genres=['edm'], limit=song_num)
    edm_uris += [i['uri'] for i in edm_recs['tracks']]
    latin_recs = sp.recommendations(seed_genres=['latin'], limit=song_num)
    latin_uris += [i['uri'] for i in latin_recs['tracks']]
    rock_recs = sp.recommendations(seed_genres=['rock'], limit=song_num)
    rock_uris += [i['uri'] for i in rock_recs['tracks']]
    randb_recs = sp.recommendations(seed_genres=['r-n-b'], limit=song_num)
    randb_uris += [i['uri'] for i in randb_recs['tracks']]
    country_recs = sp.recommendations(seed_genres=['country'], limit=song_num)
    country_uris += [i['uri'] for i in country_recs['tracks']]
    jazz_recs = sp.recommendations(seed_genres=['jazz'], limit=song_num)
    jazz_uris += [i['uri'] for i in jazz_recs['tracks']]
    classical_recs = sp.recommendations(seed_genres=['classical'], limit=song_num)
    classical_uris += [i['uri'] for i in classical_recs['tracks']]
    alternative_recs = sp.recommendations(seed_genres=['alternative'], limit=song_num)
    alternative_uris += [i['uri'] for i in alternative_recs['tracks']]

# turn into sets to remove duplicates
pop_uris = list(set(pop_uris))
hip_hop_uris = list(set(hip_hop_uris))
edm_uris = list(set(edm_uris))
latin_uris = list(set(latin_uris))
rock_uris = list(set(rock_uris))
randb_uris = list(set(randb_uris))
country_uris = list(set(country_uris))
jazz_uris = list(set(jazz_uris))
classical_uris = list(set(classical_uris))
alternative_uris = list(set(alternative_uris))

In [12]:
# compile uris
song_uris = (pop_uris + hip_hop_uris + edm_uris + latin_uris + rock_uris + randb_uris +
             country_uris + jazz_uris + classical_uris + alternative_uris)

In [13]:
# create a list for labels
genre_list = ((['pop'] * len(pop_uris)) + (['hip-hop'] * len(hip_hop_uris)) +
              (['edm'] * len(edm_uris)) + (['latin'] * len(latin_uris)) +
              (['rock'] * len(rock_uris)) + (['r-n-b'] * len(randb_uris)) +
              (['country'] * len(country_uris)) + (['jazz'] * len(jazz_uris)) +
              (['classical'] * len(classical_uris)) + (['alternative'] * len(alternative_uris)))

def create_song_dataframe(song_uris):
    '''
    combine all song URIS into a df
    input: song uris
    output: dataframe with all song features
    '''

    # set up dataframe
    col_names = ['uri', 'genre', 'key', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'valence', 'tempo']
    df = pd.DataFrame(columns=col_names) # generate empty df

    # iterate through and get features for each playlist
    for i in tqdm(range(len(song_uris))):
        # get song features
        audio_features = sp.audio_features(song_uris[i])
        key = audio_features[0]['key']
        acousticness = audio_features[0]['acousticness']
        danceability = audio_features[0]['danceability']
        energy = audio_features[0]['energy']
        instrumentalness = audio_features[0]['instrumentalness']
        loudness = audio_features[0]['loudness']
        speechiness = audio_features[0]['speechiness']
        valence = audio_features[0]['valence']
        tempo = audio_features[0]['tempo']
        features = [key, acousticness, danceability, energy, instrumentalness, loudness, speechiness, valence, tempo]
        features.insert(0, 0)
        features.insert(0, 0)
        df.loc[i] = features

    df = normalize_df(df, col_names)
    # insert uris and genres
    df['uri'] = song_uris
    df['genre'] = genre_list
    return df

In [14]:
song_df = create_song_dataframe(song_uris)
#song_df = pd.read_csv('song_df.csv', index_col=0)
song_df

  7%|▋         | 345/5231 [00:23<04:28, 18.20it/s]

In [ ]:
# label encode
for i in range(len(genres)):
    song_df['genre'] = np.where(song_df['genre'] == genres[i], i, song_df['genre'])